In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models_torch import *
import torch.optim as optim
from random import shuffle

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_160x128x128_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_160x128x128_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True, buffer=128)
valid = load_datatape(valid_tape, use_normalized_bvp=False, buffer=128)

def RandomHorizontalFlip(data, label, p=.5):
    if np.random.random()<p:
        return data[..., ::-1 ,:], label
    return data, label

def data_loader(tape, batch_size=4, shuffle_size=2000):
    shuffle_size = max(batch_size, shuffle_size)
    buff = []
    for i in tape:
        i = [i.astype(np.float16) for i in i]
        if not 40<get_hr(i[1])<180:
            continue
        buff.append(RandomHorizontalFlip(*i))
        if len(buff)>=shuffle_size:
            shuffle(buff)
            batch = [buff.pop(0) for _ in range(batch_size)]
            yield torch.tensor(np.array([i[0][..., [2, 1, 0]].transpose(3, 0, 1, 2) for i in batch])).float()*2-1, torch.tensor(np.array([i[1] for i in batch]))
    while len(buff)>=batch_size:
        shuffle(buff)
        batch = [buff.pop(0) for _ in range(batch_size)]
        yield torch.tensor(np.array([i[0][..., [2, 1, 0]].transpose(3, 0, 1, 2) for i in batch])).float()*2-1, torch.tensor(np.array([i[1] for i in batch]))

In [2]:
gra_sharp = 2.
model = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160,128,128), patches=(4,4,4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7).cuda()
lr=0.0001
step_size = 50
batch_size=4
optimizer1 = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00005)
scheduler1 = optim.lr_scheduler.StepLR(optimizer1, step_size=step_size, gamma=0.5)

criterion_Pearson = Neg_Pearson()
valid_baches = 2000

# a --> Pearson loss; b --> frequency loss
a_start = 0.1
b_start = 1
exp_b = 5

In [9]:
from fvcore.nn import FlopCountAnalysis
flops = FlopCountAnalysis(model, (torch.rand(1, 3, 160, 128, 128).cuda() ,gra_sharp)).total()
print(f'Flops per frame:{flops/160:.0f}')

Unsupported operator aten::add_ encountered 65 time(s)
Unsupported operator aten::max_pool3d encountered 3 time(s)
Unsupported operator aten::sum encountered 96 time(s)
Unsupported operator aten::add encountered 48 time(s)
Unsupported operator aten::mul encountered 24 time(s)
Unsupported operator aten::sub encountered 24 time(s)
Unsupported operator aten::div encountered 12 time(s)
Unsupported operator aten::softmax encountered 12 time(s)
Unsupported operator aten::elu encountered 26 time(s)
Unsupported operator aten::upsample_nearest3d encountered 2 time(s)
Unsupported operator aten::mean encountered 2 time(s)


Flops per frame:323798832


In [ ]:
best = np.inf
for epoch in range(10):
    loss_rPPG_avg = []
    loss_peak_avg = []
    loss_kl_avg_test = []
    loss_hr_mae = []
    model.train()
    fps = 30
    print(f'epoch:{epoch} train:')
    for i, (data, label) in enumerate(data_loader(train, batch_size=batch_size)):
        hr = torch.tensor([get_hr(i) for i in label]).float().cuda()
        data, label = data.float().cuda(), label.float().cuda()
        optimizer1.zero_grad()
        bvp, s1, s2, s3 = model(data, gra_sharp)
        bvp = (bvp-torch.mean(bvp, axis=-1).view(-1, 1))/torch.std(bvp, axis=-1).view(-1, 1)
        loss_bvp = criterion_Pearson(bvp, label)
        fre_loss = .0
        kl = .0
        train_mae = .0
        for bb in range(data.shape[0]):
            loss_distribution_kl, fre_loss_temp, train_mae_temp = cross_entropy_power_spectrum_DLDL_softmax2(bvp[bb], hr[bb], fps, std=1.0)
            fre_loss = fre_loss+fre_loss_temp
            kl = kl+loss_distribution_kl
            train_mae = train_mae+train_mae_temp
        fre_loss /= data.shape[0]
        kl /= data.shape[0]
        train_mae /= data.shape[0]
        a = a_start
        if epoch>10:
            b = 5.
        else:
            b = b_start*exp_b**(epoch/10)
        loss = a*loss_bvp + b*(fre_loss+kl)
        loss.backward()
        optimizer1.step()
        n = data.size(0)
        loss_rPPG_avg.append(float(loss_bvp.data))
        loss_peak_avg.append(float(fre_loss.data))
        loss_kl_avg_test.append(float(kl.data))
        loss_hr_mae.append(float(train_mae))
        print('epoch:%d, batch:%d, total=%d, lr=%f, sharp=%.3f, a=%.3f, NegPearson= %.4f, b=%.3f, kl= %.4f, fre_CEloss= %.4f, hr_mae= %.4f' % (epoch, i + 1, len(train)//batch_size, lr, gra_sharp, a, np.mean(loss_rPPG_avg[-2000:]), b, np.mean(loss_kl_avg_test[-2000:]), np.mean(loss_peak_avg[-2000:]), np.mean(loss_hr_mae[-2000:])), end='\r')
        if (i+1)%valid_baches==0:
            optimizer1.zero_grad()
            model.eval()
            with torch.no_grad():
                hrs = []
                for j, (data, label) in enumerate(data_loader(valid, batch_size=4)):
                    data, label = data.cuda().float(), label.cuda().float()
                    bvp, s1, s2, s3 = model(data, gra_sharp)
                    bvp = (bvp-torch.mean(bvp, axis=-1).view(-1, 1))/torch.std(bvp).view(-1, 1)
                    for _1, _2 in zip(bvp, label):
                        hrs.append((get_hr(_1.cpu().detach().numpy()), get_hr(_2.cpu().detach().numpy())))
                RMSE = np.mean([(i-j)**2 for i, j in hrs])**0.5
                print(f'Test RMSE:{RMSE:.3f}, batch:{i+1}')
                if RMSE<best:
                    best = RMSE
                    torch.save(model.state_dict(), '../weights/Physformer_CCNU.pkl')
                    print('Best model saved')
                scheduler1.step()
                if (epoch + 1) % step_size == 0:
                    lr *= 0.5

In [3]:
model.load_state_dict(torch.load('../weights/Physformer_CCNU.pkl'))
model.train()
def physformer(x, model=model):
    with torch.no_grad():
        bvp = model(torch.tensor(x[..., [2, 1, 0]].transpose(0, 4, 1, 2, 3)).cuda().float()*2-1, gra_sharp)[0].cpu().detach().numpy()
        return bvp - bvp.mean(axis=-1).reshape(-1, 1)

In [14]:
eval_on_dataset(test_set_CCNU, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysFormer_CCNU_CCNU.h5')

100%|██████████| 179/179 [36:35<00:00, 12.26s/it] 


{'Sliding window': {'MAE': 1.562, 'RMSE': 6.282, 'R': 0.80256},
 'Whole video': {'MAE': 0.674, 'RMSE': 2.859, 'R': 0.95311}}

In [5]:
#eval_on_dataset(test_set_CCNU_rPPG, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
#get_metrics('../results/PhysFormer_CCNU_CCNU_rPPG.h5')
get_metrics_HRV('../results/PhysFormer_CCNU_CCNU_rPPG.h5')

{'SDNN': {'MAE': 28.836, 'RMSE': 34.446, 'R': 0.45018}}

In [4]:
#eval_on_dataset(test_set_PURE, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
#get_metrics('../results/PhysFormer_CCNU_PURE.h5')
get_metrics_HRV('../results/PhysFormer_CCNU_PURE.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


{'SDNN': {'MAE': 21.613, 'RMSE': 32.029, 'R': 0.57577}}

In [3]:
#eval_on_dataset(test_set_UBFC_rPPG2, physformer, 160, (128, 128), step=2, batch=4, save='../results/PhysFormer_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
#get_metrics('../results/PhysFormer_CCNU_UBFC.h5')
get_metrics_HRV('../results/PhysFormer_CCNU_UBFC.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\scipy\interpolate\_fitpack2.py:298: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


{'SDNN': {'MAE': 8.365, 'RMSE': 11.149, 'R': 0.9209}}

In [9]:
#eval_on_dataset(test_set_MMPD, physformer, 160, (128, 128), step=4, batch=4, save='../results/PhysFormer_CCNU_MMPD.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysFormer_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'LED-low', 'Incandescent'])

{'Sliding window': {'MAE': 3.132, 'RMSE': 7.315, 'R': 0.68684},
 'Whole video': {'MAE': 1.696, 'RMSE': 4.135, 'R': 0.89027}}

In [4]:
eval_on_dataset(test_set_COHFACE, physformer, 160, (128, 128), step=4, batch=4, save='../results/PhysFormer_CCNU_COHFACE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysFormer_CCNU_COHFACE.h5')

100%|██████████| 164/164 [07:15<00:00,  2.66s/it]


{'Sliding window': {'MAE': 20.216, 'RMSE': 26.074, 'R': -0.29122},
 'Whole video': {'MAE': 19.986, 'RMSE': 26.077, 'R': -0.37381}}